In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.set_option("display.max_column",None)
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import re


In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import FeatureUnion

In [105]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import Lasso,Ridge,LinearRegression,ElasticNet
from sklearn.model_selection import train_test_split

In [170]:
ener = pd.read_csv("energydata_complete.csv",parse_dates= ['date'])
ener.drop(['date','lights'],axis = 1,inplace = True)

In [171]:
ener.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [16]:
ener.set_index('date',inplace=True)

In [194]:
def RSS(y_true,y_pred):
    rss = np.sum(np.square(y_true - y_pred))
    return np.round(rss,3)

In [195]:
scaler = MinMaxScaler()
koko = ener.drop(['Appliances'],axis = 1)
scaled_data = scaler.fit_transform(koko)

In [196]:
X = scaled_data
y = ener.Appliances
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [197]:
def evaluate(y_true,y_pred):
    print('Mean absolute error : ',mean_absolute_error(y_true,y_pred))
    print('RMSE : ', np.sqrt(mean_squared_error(y_true,y_pred)))
    print('Sum of squared residuals(RSS) : ', RSS(y_true,y_pred))
    print('R2_score: ',r2_score(y_true,y_pred))

# LASSO REGRESSION

In [231]:
lasso_lr = Lasso(alpha =0.001,random_state=42).fit(X_train,y_train)
lasso_pred = lasso_lr.predict(X_test)

In [232]:
lasso_lr.coef_

array([-4.14712394e+00,  5.90599827e+02, -2.49468798e+02, -4.84565333e+02,
        3.10210384e+02,  1.02204611e+02,  3.03451975e+01,  2.74459046e+01,
       -1.71187233e+01,  1.71033212e+01,  2.50080979e+02,  4.00928832e+01,
        1.03468087e+01, -4.73890941e+01,  1.08883423e+02, -1.68695476e+02,
       -2.02502545e+02, -4.24551004e+01, -3.32571693e+02,  7.28215443e+00,
       -7.80003852e+01,  3.13895144e+01,  1.30783188e+01,  1.18598030e+02,
        1.63117421e+00,  1.33691480e-15])

In [233]:
koko.columns

Index(['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5',
       'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out',
       'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1',
       'rv2'],
      dtype='object')

In [245]:
lasso_weight = pd.DataFrame(lasso_lr.coef_,index=koko.columns,columns=['Lasso'])
abs(lasso_weight)>0

,Lasso
T1,True
RH_1,True
T2,True
RH_2,True
T3,True
RH_3,True
T4,True
RH_4,True
T5,True
RH_5,True


In [246]:
lasso_weight>0

,Lasso
T1,False
RH_1,True
T2,False
RH_2,False
T3,True
RH_3,True
T4,True
RH_4,True
T5,False
RH_5,True


In [235]:
evaluate(y_test,lasso_pred)

Mean absolute error :  53.63021543531102
RMSE :  93.6406750918551
Sum of squared residuals(RSS) :  51918738.683
R2_score:  0.14889857033860587


In [229]:
evaluate(y_test,lasso_pred)

Mean absolute error :  53.63021543531102
RMSE :  93.6406750918551
Sum of squared residuals(RSS) :  51918738.683
R2_score:  0.14889857033860587


# RIDGE REGRESSION

In [204]:
ridge_lr = Ridge(alpha=0.4).fit(X_train,y_train)
ridge_pred = ridge_lr.predict(X_test)

In [205]:
ridge_weight = pd.DataFrame(ridge_lr.coef_,index=koko.columns,columns = ['Ridge'])

In [206]:
ridge_weight

,Ridge
T1,-19.694643
RH_1,555.892068
T2,-215.494498
RH_2,-439.846215
T3,308.252882
RH_3,101.794021
T4,29.300760
RH_4,26.299028
T5,-21.242949
RH_5,17.283031


In [207]:
evaluate(y_test,ridge_pred)

Mean absolute error :  53.593567049788426
RMSE :  93.66122703951963
Sum of squared residuals(RSS) :  51941531.093
R2_score:  0.14852493545092582


# Linear REGRESSION

In [208]:
Linear_lr = LinearRegression(normalize=False).fit(X_train,y_train)
Linear_pred = Linear_lr.predict(X_test)

In [209]:
linear_regression_weight = pd.DataFrame(Linear_lr.coef_,index=koko.columns,columns = ['Linear_R'])

In [210]:
linear_regression_weight.sort_values('Linear_R')

,Linear_R
RH_2,-488.666805
T_out,-344.389845
T2,-252.710373
T9,-203.236627
RH_8,-168.627161
RH_out,-83.107599
RH_7,-47.736592
RH_9,-42.586339
T5,-16.752822
T1,-3.510725


In [211]:
evaluate(y_test,Linear_pred)

Mean absolute error :  53.64297765584947
RMSE :  93.6404609399803
Sum of squared residuals(RSS) :  51918501.212
R2_score:  0.14890246319303513


# Elastic Net REGRESSION

In [146]:
elastic_lr = ElasticNet(alpha= 0.001).fit(X_train,y_train)
elastic_pred = elastic_lr.predict(X_test)

In [156]:
elastic_weight = pd.DataFrame(elastic_lr.coef_,index=koko.columns,columns=['Elastic'])

In [157]:
elastic_weight

,Elastic
Appliances,11.789443
T1,2.451878
RH_1,4.833950
T2,-4.746179
RH_2,-1.291937
T3,-3.701724
RH_3,1.055239
T4,29.769987
RH_4,17.925646
T5,-0.811573


In [150]:
evaluate(y_test,elastic_pred)

Mean absolute error :  4.81285896740205
RMSE :  6.89010753809698
Sum of squared residuals(RSS) :  281091.078
R2_score:  0.2277268290888571


In [163]:
total_weight = pd.concat([ridge_weight,lasso_weight,linear_regression_weight,elastic_weight],axis = 1)

In [80]:
rf_lr = RandomForestRegressor(n_estimators=1000).fit(X_train,y_train)
rf_pred = rf_lr.predict(X_test)

In [82]:
evaluate(y_test,rf_pred)

Mean absolute error :  2.729050836007431
RMSE :  4.780484102615787
Sum of squared residuals(RSS) :  135312.78
R2_score:  0.6282399622200374


In [79]:
evaluate(y_test,rf_pred)

Mean absolute error :  2.7443337274109103
RMSE :  4.81693411285574
Sum of squared residuals(RSS) :  137384.1
R2_score:  0.6225491923739139


In [190]:
X = ener.T2
X = scaler.fit_transform(np.array(X).reshape(-1,1))
y = ener.T6

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [191]:
Linear_lr = LinearRegression(normalize=False).fit(X_train,y_train)
Linear_pred = Linear_lr.predict(X_test)

In [192]:
evaluate(y_test,Linear_pred)

Mean absolute error :  2.8229401131313034
RMSE :  3.630315129680444
Sum of squared residuals(RSS) :  78033.972
R2_score:  0.6427789637197382


In [99]:
Linear_lr = LinearRegression(normalize=False)
Linear_lr.fit(X_,y)
Linear_lr.score(X_,y)

0.6418990830855493

In [91]:
Linear_lr = LinearRegression(normalize=False)
Linear_lr.fit(np.array(X).reshape(-1,1),y)
Linear_lr.score(np.array(X).reshape(-1,1),y)

0.6418990830855493

In [76]:
evaluate(y_test,Linear_pred)

Mean absolute error :  4.834691657669558
RMSE :  6.885552366298004
Sum of squared residuals(RSS) :  280719.533
R2_score:  0.2287476184155015


In [167]:
ener

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,7.026667,84.256667,17.200000,41.626667,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,6.833333,84.063333,17.200000,41.560000,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,6.560000,83.156667,17.200000,41.433333,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,6.433333,83.423333,17.133333,41.290000,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,6.366667,84.893333,17.200000,41.230000,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,24.796667,1.000000,24.500000,44.500000,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,24.196667,1.000000,24.557143,44.414286,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,23.626667,1.000000,24.540000,44.400000,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
